<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Creating PostgreSQL Servers and Databases

## Azure Actions Covered

* Creating a PostgreSQL server
* Creating a firewall rule for a PostgreSQL database
* Creating a database on a PostgreSQL server
* Connecting to a PostgreSQL server

In this lecture, we'll learn how to create PostgreSQL servers and databases on Azure with Python.

To begin, we'll need to import our usual libraries as well as any useful environment variables (e.g. AZURE_SUBSCRIPTION_ID). We also need to import some modules to work with the PostgreSQL flexible servers.

In [1]:
from azure.identity import AzureCliCredential
# New imports
from azure.mgmt.rdbms.postgresql_flexibleservers import PostgreSQLManagementClient
from azure.mgmt.rdbms.postgresql_flexibleservers.models import Server, Sku, Storage

# For connection to PostgreSQL database
import psycopg2

from settings import AZURE_SUBSCRIPTION_ID, DEFAULT_LOCATION, DEFAULT_RESOURCE_GROUP

Instantiate your Azure credential and then your `PostgreSQLManagementClient`.

In [2]:
credential = AzureCliCredential()
postgres_client = PostgreSQLManagementClient(credential, AZURE_SUBSCRIPTION_ID)

## Creating a Server

We can use the PostgreSQL client to create a new server. We'll need the following options:

* `resource_group_name` - Name of the resource group where the server will be created
* `server_name` - Name for the Postgres server
* `parameters` - Parameters for Postgres server creation, including:
    * `location` - Azure location for server
    * `administrator_login` - User name for admin user
    * `adminsitrator_login_password` - Password for admin user

For a full list of parameters, see the [Server class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-rdbms/azure.mgmt.rdbms.postgresql_flexibleservers.models.server?view=azure-python). For other parameters/classes, see:

* [Sku class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-rdbms/azure.mgmt.rdbms.postgresql_flexibleservers.models.sku?view=azure-python)
* [Storage class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-rdbms/azure.mgmt.rdbms.postgresql_flexibleservers.models.storage?view=azure-python)

In [3]:
poller = postgres_client.servers.begin_create(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    parameters=Server(
        location=DEFAULT_LOCATION,
        administrator_login='benadmin',
        administrator_login_password='testpassword123!',
        sku=Sku(name='Standard_D2s_v3', tier='GeneralPurpose'),
        storage=Storage(storage_size_gb=32),
        version='13'
    )
)
server = poller.result()

We can check that our server was created by checking the `name` attribute on the returned server object.

In [4]:
server.name

'bens-server'

Next, we need to create a firewall rule to allow traffic to our server. For a full list of possible firewall rule parameters, see the [FirewallRule class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-rdbms/azure.mgmt.rdbms.postgresql_flexibleservers.models.firewallrule?view=azure-python)

In [5]:
poller = postgres_client.firewall_rules.begin_create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    firewall_rule_name='AllowMyIp',
    parameters={
        'start_ip_address': '72.174.17.20',
        'end_ip_address': '72.174.17.20'
    }
)
firewall_rule = poller.result()

Again, to check that it was created we can use the `name` attribute on the returned object.

In [6]:
firewall_rule.name

'AllowMyIp'

Finally, as we've seen with other clients, we can list all of our servers as well.

In [7]:
for server in postgres_client.servers.list():
    print(server.name)

bens-server


## Creating a Database

Let's create a new Postgres database. We'll need:

* `resource_group_name` - Name of the resource group where the DB will be created
* `server_name` - Server where the DB will be created
* `database_name` - Name for the new database
* `parameters` - Parameters for DB creation. For full list, see the [Database class](https://learn.microsoft.com/en-us/python/api/azure-mgmt-rdbms/azure.mgmt.rdbms.postgresql_flexibleservers.models.database?view=azure-python)

We won't pass any parameters because the default creation parameters are fine for now.

In [9]:
poller = postgres_client.databases.begin_create(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    database_name='bens-db',
    parameters={}
)
db = poller.result()

Let's check the database name on the result.

In [10]:
db.name

'bens-db'

We can also list all of the databases on our server using the `list_by_server()` method. You'll see some system databases as well as the one we just created.

In [11]:
for db in postgres_client.databases.list_by_server(DEFAULT_RESOURCE_GROUP, 'bens-server'):
    print(db.name)

azure_maintenance
postgres
azure_sys
bens-db


## Connecting to the Server and Creating a Firewall Rule

Before we connect to the Postgres server, we may need a new firewall rule to allow connections from our Jupyter Notebook.

In [26]:
firewall = sql_client.firewall_rules.create_or_update(
    resource_group_name=DEFAULT_RESOURCE_GROUP,
    server_name='bens-server',
    firewall_rule_name='JupyterNotebook',
    parameters={
        'start_ip_address': '72.175.52.98',
        'end_ip_address': '72.175.52.98'
    }
)


Now, let's use `psycopg2` to connect to our server. We need the following parameters:

* `host` - Host server. This will be of the form `<server-name>.postgres.database.azure.come`
* `database` - Name of database to connect to
* `user` - User name, we can use our admin user for now
* `password` - Password for the user, we can use our admin user's password

In [13]:
connection = psycopg2.connect(
    host='bens-server.postgres.database.azure.com',
    database='bens-db',
    user='benadmin',
    password='testpassword123!'
)

We'll also need a cursor to execute queries, so let's instantiate that in a variable.

In [14]:
cursor = connection.cursor()

Let's execute a quick query on the `information_schema` for our database.

In [16]:
cursor.execute("""
    select table_name, table_schema
    from information_schema.tables
    limit 15;
""")
rows = cursor.fetchall()

In [17]:
rows

[('pg_statistic', 'pg_catalog'),
 ('pg_type', 'pg_catalog'),
 ('pg_foreign_table', 'pg_catalog'),
 ('pg_roles', 'pg_catalog'),
 ('pg_settings', 'pg_catalog'),
 ('pg_hba_file_rules', 'pg_catalog'),
 ('pg_config', 'pg_catalog'),
 ('pg_shmem_allocations', 'pg_catalog'),
 ('pg_prepared_statements', 'pg_catalog'),
 ('pg_stat_progress_vacuum', 'pg_catalog'),
 ('pg_replication_origin_status', 'pg_catalog'),
 ('pg_subscription', 'pg_catalog'),
 ('pg_attribute', 'pg_catalog'),
 ('pg_proc', 'pg_catalog'),
 ('pg_class', 'pg_catalog')]

Finally, we'll close our cursor and our connection to the database.

In [18]:
cursor.close()
connection.close()